In [ ]:
#####################
using CSV
using Dates, DataFrames
using NativeFileDialog
using Plots, Printf

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# select directory
csv_directory = pick_folder()

# build list of all csv files in selected directory
csv_files = filter(x->occursin(".csv",x), readdir(csv_directory));

#try

    # select the 0xf20 file
    f20_file_name = csv_files[findfirst(contains("{0xF20}"),csv_files[findall(x->endswith(uppercase(x), ".CSV"), csv_files)])];

    f20_file = joinpath(csv_directory,f20_file_name)

    # create a df to hold the f20 data
    f20_df = DataFrame(CSV.File(f20_file,header=0, delim="\t"));

# select the 0xf21 file
    f21_file_name = csv_files[findfirst(contains("{0xF21}"),csv_files[findall(x->endswith(uppercase(x), ".CSV"), csv_files)])];

    f21_file = joinpath(csv_directory,f21_file_name)

    # create a df to hold the f21 data
    f21_df = DataFrame(CSV.File(f21_file,header=0, delim="\t"));
#end

f20_df_copy = copy(f20_df)
f21_df_copy = copy(f21_df);

In [ ]:
f20_df = copy(f20_df_copy)
f21_df = copy(f21_df_copy);

In [ ]:
Spectra = []
Direction = []
Spread = []

# Process Heave spectrum message (0xF20)
for i in 1:nrow(f20_df)
    push!(Spectra,Array(f20_df[i, 4:103])')
end

f20_df.Spectra = Spectra

[select!(f20_df, Not(j)) for j in ["Column$i" for i in 4:103]]
    
# convert Epoch seconds to UTC
insertcols!(f20_df, 1,  :Date =>  unix2datetime.(f20_df.Column1))

[select!(f20_df, Not(j)) for j in ["Column$i" for i in 1:2]]
    
# Process Primary directional spectrum message (0xF21)
for i in 1:nrow(f21_df)
    push!(Direction,Array(f21_df[i, 4:103])')
    push!(Spread,Array(f21_df[i, 104:203])')
end

f21_df.Direction = Direction
f21_df.Spread = Spread

[select!(f21_df, Not(j)) for j in ["Column$i" for i in 4:203]];
            
# convert Epoch seconds to UTC
insertcols!(f21_df, 1,  :Date =>  unix2datetime.(f21_df.Column1))

[select!(f21_df, Not(j)) for j in ["Column$i" for i in 1:3]];
                

In [ ]:
merged_df = innerjoin(f20_df, f21_df, on = :Date)

In [ ]:
p1 = Plots.plot()

[p1 = Plots.plot!(merged_df.Spectra[i]', label="") for i in 1:20]

display(p1)